In [1]:
import random
from dash import Dash, html, dcc, Input, Output, State, ctx

app = Dash(__name__)

def choose_word():
    """Choose a random word from a predefined list of animals."""
    words_list = ["elephant", "giraffe", "dolphin", "kangaroo", "rhinoceros", "crocodile", "chimpanzee", "penguin"]
    return random.choice(words_list).lower()

def update_guessed_word(chosen_word, guessed_letters):
    """Update the displayed word with correctly guessed letters."""
    return [letter if letter in guessed_letters else "_" for letter in chosen_word]

# Initialize the game state
chosen_word = choose_word()
guessed_word = ["_"] * len(chosen_word)
attempts_remaining = 6
guessed_letters = []

app.layout = html.Div([
    html.H1("Welcome to the Hangman Game!"),
    html.Div(id='word-display', style={'fontSize': '24px'}),
    html.Div(id='attempts-remaining', style={'fontSize': '20px'}),
    html.Div(id='guessed-letters', style={'fontSize': '20px'}),
    dcc.Input(id='letter-input', type='text', maxLength=1, placeholder='Guess a letter'),
    html.Button('Submit', id='submit-button', n_clicks=0),
    html.Div(id='result'),
])

@app.callback(
    Output('word-display', 'children'),
    Output('attempts-remaining', 'children'),
    Output('guessed-letters', 'children'),
    Output('result', 'children'),
    Input('submit-button', 'n_clicks'),
    State('letter-input', 'value'),
    prevent_initial_call=True
)
def update_game(n_clicks, letter):
    global guessed_word, attempts_remaining, guessed_letters, chosen_word

    if letter and len(letter) == 1 and letter.isalpha():
        letter = letter.lower()

        if letter in guessed_letters:
            return " ".join(guessed_word), f"Attempts remaining: {attempts_remaining}", "Guessed letters: " + " ".join(sorted(guessed_letters)), "You've already guessed this letter."
        
        guessed_letters.append(letter)

        if letter in chosen_word:
            guessed_word = update_guessed_word(chosen_word, guessed_letters)
        else:
            attempts_remaining -= 1
        
        if "_" not in guessed_word:
            return " ".join(guessed_word), f"Attempts remaining: {attempts_remaining}", "Guessed letters: " + " ".join(sorted(guessed_letters)), f"Congratulations! You've guessed the animal: {''.join(guessed_word)}"
        
        if attempts_remaining <= 0:
            return " ".join(guessed_word), f"Attempts remaining: {attempts_remaining}", "Guessed letters: " + " ".join(sorted(guessed_letters)), f"Sorry! You've run out of attempts. The animal was: {chosen_word}"

    return " ".join(guessed_word), f"Attempts remaining: {attempts_remaining}", "Guessed letters: " + " ".join(sorted(guessed_letters)), ""

if __name__ == '__main__':
    app.run_server(debug=True)


# http://127.0.0.1:8050/